<a href="https://colab.research.google.com/github/gvogiatzis/CS3320/blob/main/CS3320_Lab_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS3320 Lab 2. Ranked Retrieval